# Self-Driving Car Engineer Nanodegree

## Project: **Finding Lane Lines on the Road** 

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
import matplotlib.image as mpimg
import numpy as np
import laneLines
%matplotlib inline

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
pl = laneLines.Pipeline()
for fname in os.listdir("test_images/"):
    img = mpimg.imread('test_images/%s' % fname)
    pl.prettyShow(img)

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [ ]:
pl = laneLines.Pipeline()
def process_image(image, savepath=None):
    ip = pl(image)
    if savepath is not None:
        laneLines.saveImage(ip, savepath)
    return ip

In [ ]:
for fname in os.listdir("test_images/"):
    img = mpimg.imread('test_images/%s' % fname)
    savepath = 'test_images_output/%s' % fname
    ip = process_image(img, savepath)

## Test on Videos

Let's try the one with the solid white lane on the right first ...

In [ ]:
pl = laneLines.Pipeline(horizon=.6, min_line_len=42)
pl.processVideo(
    'test_videos/solidWhiteRight.mp4', 
    'test_videos_output/solidWhiteRight.mp4'
)

## Improve the draw_lines() function

At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".

Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
from importlib import reload; reload(laneLines)
pl = laneLines.Pipeline(
    min_line_len=42, horizon=.6, 
    theta=np.pi/180, houghThreshold=64
)
pl.processVideo(
    'test_videos/solidYellowLeft.mp4',
    'test_videos_output/solidYellowLeft.mp4', 
    subsection=(0, 1),
)

In [ ]:
from moviepy.editor import VideoFileClip
pl.prettyShow(VideoFileClip('test_videos/solidYellowLeft.mp4').get_frame(1));

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
pl = laneLines.Pipeline(hoodClearance=.1, horizonRadius=.04, min_line_len=32, nsmooth=24)
#pl.debug = True
pl.processVideo(
    'test_videos/challenge.mp4',
    'test_videos_output/challenge.mp4'
)

In [ ]:
from importlib import reload
reload(laneLines)
from moviepy.editor import VideoFileClip
pl = laneLines.Pipeline(horizon=.6, hoodClearance=.1, horizonRadius=.04, min_line_len=32)

frame = VideoFileClip('test_videos/challenge.mp4').get_frame(5.05)
fig, ax = pl.prettyShow(frame)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
l2p = lambda ls: np.stack([
        (l.m, l.b)
        for l in ls
        if np.abs(l.m) > .5
    ])
prepped = pl.prepare(frame)
lines = pl.findLines(prepped)
linesdedup = pl.deduplicate_lines(lines)

fig, ax1 = plt.subplots()
ax1.imshow(prepped)
k = 0
for l in lines[k:k+1]:
    l.plotline(ax1, lw=8, alpha=.5)
for l in linesdedup:
    l.plotline(ax1, lw=42, alpha=.5, color='green', linestyle='--')

fig, ax2 = plt.subplots()
ax2.scatter(l2p(lines)[:, 0], l2p(lines)[:, 1])
ax2.scatter(l2p(linesdedup)[:, 0], l2p(linesdedup)[:, 1], edgecolor='red', s=96, facecolor='none', marker='*')
ax2.set_xlabel('slope $m$'); ax2.set_ylabel('intercept $b$')